# Amazon Web Scraping

In [103]:
#import nessary libraries

from bs4 import BeautifulSoup
import requests
import pandas as pd


Functions to extract values in each pages

In [104]:
# Function to extract title 
def get_title(soup):
    try:
        #outer tag object
        title=soup.find("span",attrs={"id":'productTitle'}).get_text().strip()

    except AttributeError:
        title=""
    return title

#Function to extract Price
def get_price(soup):
    try:
        price=soup.find('span',attrs={"class":"a-offscreen"}).get_text().strip()[1:]
        # price=float(price)
    except AttributeError:
        price=''
    return price


#Function to extract ratings
def get_star(soup):
    try:
        star=soup.find("i",attrs={"class":"a-icon a-icon-star a-star-4-5 cm-cr-review-stars-spacing-big"}).get_text().strip().split()[0]
        # star=float(star)
    except AttributeError:
        star=''
    return star

#Function to extract review counts
def get_reviews(soup):
    try:
        review=soup.find("span",attrs={"id":"acrCustomerReviewText"}).get_text().strip().split()[0]
        # review=float(review)
    except AttributeError:
        review=''
    return review

#Function to extract availability
def get_availability(soup):
    try:
        availability=soup.find("div",attrs={"id":"availability"}).get_text().strip()
    except AttributeError:
        availability=''
    return availability


Creating Csv file with headers

In [106]:

#url of the desired page
URL= "https://www.amazon.com/s?k=tshirt&crid=12L97UNZ3NXOS&sprefix=tsh%2Caps%2C1716&ref=nb_sb_noss_2"

#headers for the requests
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

#Http Request
page = requests.get(URL, headers=headers)           #bring data to the local environment

#Soup objects containing all data as html 
soup1 = BeautifulSoup(page.content, "html.parser")         #retrieve data as html contents
soup2 = BeautifulSoup(soup1.prettify(), "html.parser")     #showcase html data as better indentation


#fetch link as list of tag objects
links=soup2.find_all('a',attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})    #finding all anchor tags to extract url links in the web page. this gives as a block

links_list=[]
for link in links:
    links_list.append(link.get('href'))

#making empty dictionary to convert as dataframe
d={"title":[],"price":[],"rating":[],"reviews":[],"availability":[]}

#modify each link as web brower links to achieve each item web pages
for link in links_list:
    new_webpage=requests.get("https://www.amazon.com"+link,headers=headers)
    new_soup=BeautifulSoup(new_webpage.content,'html.parser')
    new_soup2=BeautifulSoup(new_soup.prettify(),'html.parser')
    get_title(soup2)

    
    #appending data to the empty dictionary
    d['title'].append(get_title(new_soup2))
    d['price'].append(get_price(new_soup2))
    d['rating'].append(get_star(new_soup2))
    d['reviews'].append(get_reviews(new_soup2))
    d['availability'].append(get_availability(new_soup2))

        

print(page)

<Response [200]>


Data Frames with Pandas

In [107]:
#converet dictionary to datafram
amazon_df=pd.DataFrame.from_dict(d)
pd.set_option('display.max_rows',None)

#convert dataframe to csv
amazon_df.to_csv('Amazon.csv')

#reading csv in pandas
df=pd.read_csv("./Amazon.csv").head()


Data Cleaning in Pandas

In [108]:
import datetime

#Handling Null values 
df['availability'].fillna("out of stocks",inplace=True)

#drop unnessary columns
df = df.drop('Unnamed: 0', axis=1)
# df
today=datetime.date.today()
df['Date']=today

df

,title,price,rating,reviews,availability,Date
0,Amazon Essentials Women's Classic-Fit Short-Sl...,19.00,4.4,"46,394",In Stock,2023-06-14
1,"Gildan Men's Crew T-Shirts, Multipack, Style G...",19.99,4.6,"260,382",out of stocks,2023-06-14
2,5 Pack Men’s Active Quick Dry Crew Neck T Shir...,39.95,4.6,"32,528",out of stocks,2023-06-14
3,"Hanes Mens Beefyt T-Shirt, Classic Heavyweight...",6.79,4.5,"58,648",out of stocks,2023-06-14
4,Fruit of the Loom Men's Eversoft Cotton Short ...,21.38,4.5,"52,906",out of stocks,2023-06-14
